In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [3]:
EPOCHS = 20
BATCH_SIZE = 64

In [4]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [5]:
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    download = True,
    transform = transform
)

testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    download = True,
    transform = transform
)

In [6]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = batch_size)

test_loader = torch.utils.data.DataLoader(
    dataset = testset,
    batch_size = batch_size)

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [16]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [17]:
model.train()
optimizer.zero_grad()

In [18]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [19]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [20]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss : 0.5575, Accuracy : 80.45%
[2] Test Loss : 0.4895, Accuracy : 82.36%
[3] Test Loss : 0.4514, Accuracy : 83.79%
[4] Test Loss : 0.4237, Accuracy : 84.76%
[5] Test Loss : 0.4047, Accuracy : 85.46%
[6] Test Loss : 0.3904, Accuracy : 85.93%
[7] Test Loss : 0.3811, Accuracy : 86.50%
[8] Test Loss : 0.3737, Accuracy : 86.64%
[9] Test Loss : 0.3683, Accuracy : 86.87%
[10] Test Loss : 0.3627, Accuracy : 86.89%
[11] Test Loss : 0.3591, Accuracy : 87.19%
[12] Test Loss : 0.3535, Accuracy : 87.28%
[13] Test Loss : 0.3507, Accuracy : 87.52%
[14] Test Loss : 0.3479, Accuracy : 87.60%
[15] Test Loss : 0.3465, Accuracy : 87.61%
[16] Test Loss : 0.3449, Accuracy : 87.77%
[17] Test Loss : 0.3412, Accuracy : 87.83%
[18] Test Loss : 0.3428, Accuracy : 87.82%
[19] Test Loss : 0.3371, Accuracy : 88.05%
[20] Test Loss : 0.3352, Accuracy : 88.12%
